### Setting up path folder

In [1]:
PARENT_FOLDER='notebooks'

import os, sys

this_path = os.getcwd()
folders = this_path.split('/')
index = folders.index(PARENT_FOLDER)
steps_to_root = len(folders) - folders.index(PARENT_FOLDER)
src_path = "/".join(['..'] * steps_to_root)

if src_path not in sys.path:
    sys.path.append(src_path)

### Importing libraries

In [2]:
import source.sql_agent as sqa

In [30]:
db_uri="sqlite:///Chinook.db"
db_dialect = "SQLite"
a = sqa.SQL_Agent(model_provider="openai",model_name="gpt-4o-mini",database_uri=db_uri, dialect=db_dialect, top_k=5)
#a = sqa.SQL_Agent(model_provider="groq",model_name="llama3-8b-8192",database_uri=db_uri, dialect=db_dialect, top_k=5)
#a = sqa.SQL_Agent(model_provider="groq",model_name="llama-3.3-70b-versatile",database_uri=db_uri, dialect=db_dialect, top_k=5)

/Users/emalaga/Dropbox/Ed Work/Coding Projects/Tutorials/.venv/lib/python3.11/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
question = "What country has the biggest sales?"
a.stream_answer(question)

================================ Human Message =================================

What country has the biggest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_VOzq5fZWOV3Av7E4SviP2pOe)
 Call ID: call_VOzq5fZWOV3Av7E4SviP2pOe
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_NcXeN4mN9ZJPXZ0QZzuFzMmm)
 Call ID: call_NcXeN4mN9ZJPXZ0QZzuFzMmm
  Args:
    table_names: Invoice
  sql_db_schema (call_n5a1OtKteS7eRlvZWcptLptA)
 Call ID: call_n5a1OtKteS7eRlvZWcptLptA
  Args:
    table_names: Customer
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"Customer

In [4]:
question = "Which album has the most tracks?"
a.stream_answer(question)

================================ Human Message =================================

Which album has the most tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_FcGZo623ExLnZIbCUMxSi4Ru)
 Call ID: call_FcGZo623ExLnZIbCUMxSi4Ru
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_hae67LR63u5S0AVY7vEbdnoh)
 Call ID: call_hae67LR63u5S0AVY7vEbdnoh
  Args:
    table_names: Album
  sql_db_schema (call_mDXdhKljHmwSN7ZSNIRe5ZdN)
 Call ID: call_mDXdhKljHmwSN7ZSNIRe5ZdN
  Args:
    table_names: Track
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Track" (
	"TrackId" INTEGER NO

In [7]:
question = "What Album has sold the most tracks?"
a.stream_answer(question)

================================ Human Message =================================

What Album has sold the most tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_wYt8aznxvvPF47agtA39cTxr)
 Call ID: call_wYt8aznxvvPF47agtA39cTxr
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_KEG8lhTaQbqH66FXYkepvm42)
 Call ID: call_KEG8lhTaQbqH66FXYkepvm42
  Args:
    table_names: Album
  sql_db_schema (call_WcB9ZLyRbGQ9OuZ7GCLW1MEd)
 Call ID: call_WcB9ZLyRbGQ9OuZ7GCLW1MEd
  Args:
    table_names: Track
  sql_db_schema (call_GBUpVYUSqDlYv7UPWzvUEHaX)
 Call ID: call_GBUpVYUSqDlYv7UPWzvUEHaX
  Args:
    table_names: InvoiceLine
==================

In [ ]:
question = " Which artist has sold the most tracks?"
a.stream_answer(question)

================================ Human Message =================================

 Which artist has sold the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_LgMk4JY0C8bK9rZBAsdlQpmm)
 Call ID: call_LgMk4JY0C8bK9rZBAsdlQpmm
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_u54guHkw9va14iXfHwpnsFZh)
 Call ID: call_u54guHkw9va14iXfHwpnsFZh
  Args:
    table_names: Artist, Invoice, InvoiceLine
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
Art

In [27]:
question = "What is the average duration of the top 5 selling tracks?"
a.stream_answer(question)

================================ Human Message =================================

What is the average duration of the top 5 selling tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kwmp)
 Call ID: call_kwmp
  Args:
    tool_input: 
  sql_db_schema (call_2qrx)
 Call ID: call_2qrx
  Args:
    table_names: table1, table2, table3
  sql_db_query (call_n3q3)
 Call ID: call_n3q3
  Args:
    query: SELECT duration FROM table1 WHERE duration IS NOT NULL ORDER BY duration DESC LIMIT 5
================================= Tool Message =================================
Name: sql_db_query

Error: (sqlite3.OperationalError) no such table: table1
[SQL: SELECT duration FROM table1 WHERE duration IS NOT NULL ORDER BY duration DESC LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_w124)
 Cal

In [ ]:
custom_prompt = 'System: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nWhen writing joins you must double check that you are using the same column name for both tables.\nNEVER join columns that have different names.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the database to see what you can query.\nDo NOT skip this step.\nThen you should query the schema of the most relevant tables.'
a.set_custom_prompt(dialect='SQLite', top_k=5,custom_prompt=custom_prompt)a.stream_answer(question)

================================ Human Message =================================

what percent of customers have bought tracks from the top 2 albums with the highest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_21SIWRdEgOlC2VhIcvXJltE1)
 Call ID: call_21SIWRdEgOlC2VhIcvXJltE1
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_lC5SKFYfuQkumd6BMlxIb9KG)
 Call ID: call_lC5SKFYfuQkumd6BMlxIb9KG
  Args:
    table_names: Album, Customer, Invoice, InvoiceLine
  sql_db_schema (call_tmfv3PGYBT5GT1VtBaXr7jhd)
 Call ID: call_tmfv3PGYBT5GT1VtBaXr7jhd
  Args:
    table_names: Track
================================= Tool Message ==========

In [22]:


custom_prompt = '''System: You are an agent designed to interact with a SQL database.

Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. 

Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for the relevant columns given the question.

When writing joins you must double check that you are using the same column name for both tables.

NEVER join columns that have different names. All columns in foreign keys have the same name. 

You have access to tools for interacting with the database.

Only use the below tools. Only use the information returned by the below tools to construct your final answer.

You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

In this database, sales are calculated by multiplying UnitPrice by Quantity from the InvoiceLine table. This is the only way to calculate sales. 

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables.
'''

a.set_custom_prompt(dialect='SQLite', top_k=5,custom_prompt=custom_prompt)
a.initialize_agent()

In [32]:
#question = "what percent of customers have bought tracks from the top 2 albums with the highest sales?"
question = "what are the sales for the top 5 selling albums?"
a.stream_answer(question)

================================ Human Message =================================

what are the sales for the top 5 selling albums?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_bPoRVbu05xKZwyRm9gc9lLqc)
 Call ID: call_bPoRVbu05xKZwyRm9gc9lLqc
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_ewMlOKs8FuwO7WEq2uqixfWw)
 Call ID: call_ewMlOKs8FuwO7WEq2uqixfWw
  Args:
    table_names: Album
  sql_db_schema (call_v4K7sZz4BchwyHMLBgkVOycR)
 Call ID: call_v4K7sZz4BchwyHMLBgkVOycR
  Args:
    table_names: InvoiceLine
  sql_db_schema (call_zi6MyCpKfuMj9L701u0i2tee)
 Call ID: call_zi6MyCpKfuMj9L701u0i2tee
  Args:
    table_names: Track
======

In [35]:
question = "Defining sales as the product of quantity by unit price, what percent of customers have bought tracks from the 2 albums with the higest sales?"
a.stream_answer(question)

================================ Human Message =================================

Defining sales as the product of quantity by unit price, what percent of customers have bought tracks from the 2 albums with the higest sales?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_qiOTpFFXJngFX3GUdGrOxdC2)
 Call ID: call_qiOTpFFXJngFX3GUdGrOxdC2
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_sZ3uEm0CUs80qDGTldK6ud2D)
 Call ID: call_sZ3uEm0CUs80qDGTldK6ud2D
  Args:
    table_names: Album, Invoice, InvoiceLine, Track, Customer
  sql_db_schema (call_TmPBXB5yeFSrl18vJRPItmM3)
 Call ID: call_TmPBXB5yeFSrl18vJRPItmM3
  Args:
    table_names: Invo